In [9]:
import logging
import requests
from bs4 import BeautifulSoup
import re
import json
import urllib.parse
from concurrent.futures import ThreadPoolExecutor

In [10]:
# URL страницы
url = "https://www.hplovecraft.com/life/friends.aspx"

# Получаем HTML-код страницы
response = requests.get(url)
response.raise_for_status()  # Проверка успешности запроса
soup = BeautifulSoup(response.text, 'html.parser')

# Регулярные выражения для поиска имени и дат
name_pattern = re.compile(r"^[A-Za-z\s\.\-'\[\]]+")
date_pattern = re.compile(r"\((b\.\s?\d{4}|\d{4})?–?(still_alive|\d{4}|unknown)?\)")
alias_date_pattern = re.compile(r"\(.*?,\s(\d{4})–(\d{4})\)")

# Результат
friends = []

# Находим все абзацы с жирным текстом (имена друзей Лавкрафта)
paragraphs = soup.find_all('p')

for para in paragraphs:
    bold_text = para.find('b')
    if not bold_text:
        continue  # Пропускаем абзацы без жирного текста

    text = para.get_text().strip()

    # Извлечение имени
    name_match = name_pattern.match(bold_text.text)
    if not name_match:
        continue
    name = name_match.group(0).strip()
    name = re.sub(r"\[.*?\]", "", name)  # Убираем квадратные скобки и их содержимое

    # Извлечение дат (основной случай)
    birth_date = "unknown"
    death_date = "unknown"
    date_match = date_pattern.search(text)
    if date_match:
        birth_part = date_match.group(1)
        if birth_part:
            if "b." in birth_part:
                birth_date = birth_part.replace("b.", "").strip()
            else:
                birth_date = birth_part
        death_date = date_match.group(2) if date_match.group(2) else "unknown"

    # Извлечение дат из формата с псевдонимом
    alias_date_match = alias_date_pattern.search(text)
    if alias_date_match:
        birth_date = alias_date_match.group(1)
        death_date = alias_date_match.group(2)

    # Описание (оставшаяся часть текста после дат)
    description_start = text.find("),") + 2 if "), " in text else len(name)
    content = text[description_start:].strip()

    # Приведение описания к началу с большой буквы
    if content:
        content = content[0].upper() + content[1:]

    # Добавляем данные в список
    friends.append({
        "title": name,
        "class": "RealWorldPerson",
        "subclass": "LovecraftSFriendsAndAcquaintances",
        "content": content,
        "infobox": {
            "birth_date": birth_date,
            "death_date": death_date,
            "mate": "H. P. Lovecraft"
        }
    })

# Сохраняем результат в JSON-файл
output_file = "lovecraft_friends.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(friends, f, ensure_ascii=False, indent=4)

print(f"Данные успешно сохранены в файл {output_file}")

Данные успешно сохранены в файл lovecraft_friends.json


In [11]:
def clean_content(content):
    """Удаляет содержимое в первой паре скобок и двоеточие после них."""
    return re.sub(r"^\(.*?\):\s*", "", content)
    
def parse_family_page(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, "html.parser")

    relatives = []
    family_list = soup.find_all("li")  # Находим все <li>

    for relative in family_list:
        bold_name = relative.find("b")
        if bold_name:
            title = bold_name.text.strip()  # Извлекаем имя

            # Извлекаем даты рождения и смерти
            birth_date, death_date = None, None
            date_match = re.search(r"\((\d{4})(?:–|—)([\w\s\d]+)?\):", relative.text)
            if date_match:
                birth_date = date_match.group(1)
                death_date = date_match.group(2)

            # Получаем контент, ограниченный текущим <li>
            content = relative.decode_contents()  # Получаем HTML содержимое <li>
            # Извлекаем контент начиная с двоеточия после скобок
            content_match = re.search(r"\):\s*(.*?)(?=<li>|$)", str(relative), re.DOTALL)
            content_text = clean_content(content_match.group(1).strip() if content_match else "")

            # Создаем инфобокс
            infobox = {
                "birth_date": birth_date,
                "death_date": death_date,
                "isRelativeTo": "H. P. Lovecraft"
            }

            # Добавляем данные в список
            relatives.append({
                "title": title,
                "class": "RealWorldPerson",
                "subclass": "LovecraftSRelatives",
                "content": content_text,
                "infobox": infobox
            })

    return relatives

# URL страницы
url = "https://www.hplovecraft.com/life/family.aspx"
relatives_data = parse_family_page(url)

# Сохранение результата в JSON
output_file = "lovecraft_relatives.json"
with open(output_file, "w", encoding="utf-8") as file:
    json.dump(relatives_data, file, ensure_ascii=False, indent=4)

print(f"Парсинг завершён. Данные сохранены в '{output_file}'")

2025-01-14 15:01:02 - DEBUG - Encoding detection: ascii is most likely the one.


Парсинг завершён. Данные сохранены в 'lovecraft_relatives.json'


In [8]:
# Настройка логирования
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.getLogger("urllib3").setLevel(logging.CRITICAL)

BASE_URL = "https://lovecraft.fandom.com"

classification_keywords = {
    "Outer God": ["Outer God", "Outer Gods"],
    "Elder God": ["Elder God", "Elder Gods"],
    "Great Old One": ["Great Old One", "Great Old Ones"],
    "Great One": ["Great One", "Great Ones"]
}

relative_keywords = {
    "parent": "hasParent",
    "grandparent": "hasGrandParent",
    "greatgrandparent": "hasGreatGrandParent",
    "father": "hasFather",
    "mother": "hasMother",
    "sister": "hasSister",
    "brother": "hasBrother",
    "halfsister": "hasHalfSister",
    "halfbrother": "hasHalfBrother",
    "son": "hasSon",
    "daughter": "hasDaughter",
    "offspring": "hasOffspring",
    "ancestor": "hasAncestor",
    "uncle": "hasUncle",
    "cousin": "hasCousin",
    "nephew": "hasNephew",
    "grandfather": "hasGrandfather",
    "grandmother": "hasGrandmother",
    "grandson": "hasGrandson",
    "granddaughter": "hasGranddaughter",
    "greatgrandson": "hasGreatGrandson",
    "greatgrandfather": "hasGreatGrandFather",
    "greatgreatgrandson": "hasGreatGreatGrandson",
    "greatgreatgrandfather": "hasGreatGreatGrandfather",
    "greatgranddaughter": "hasGreatGranddaughter",
    "greatgrandchildren": "hasGreatGrandchildren",
    "greatgreatgranddaughter": "hasGreatGreatGranddaughter",
    "sibling": "hasSibling",
    "relative": "hasRelative"
}

EXCLUDED_CATEGORIES = ["Article stubs", "stub", "Pages needing cleanup", "Unsorted", "Articles needing citations",
                      "Gallery Pages", "Clean-up", "Browse", "2 more", "3 more", "4 more", "5 more", "6 more", "7 more", "9 more",
                      "Candidates for deletion", "Candidates for merging", "Pages using ISBN magic links", "Link Story Dev", 
                       "Needs more Source Analysis"]

def normalize_relation(relation):
    return relation.lower().replace(" ", "").replace("-", "").strip()

def clean_categories(categories):
    return list({cat for cat in categories if cat not in EXCLUDED_CATEGORIES})

def extract_categories(soup):
    """
    Извлекает все категории, включая скрытые за 'X more', исключая лишние элементы вроде 'X more'.
    """
    categories = []
    
    # Основные категории
    categories_elem = soup.find("div", class_="page-header__categories")
    if categories_elem:
        # Извлекаем текст из всех ссылок
        categories.extend([link.text.strip() for link in categories_elem.find_all("a")])

    # Дополнительные категории из всех выпадающих меню
    dropdown_contents = soup.find_all("div", class_="wds-dropdown__content")
    for dropdown_content in dropdown_contents:
        # Извлекаем текст из дополнительных категорий
        more_categories = [link.text.strip() for link in dropdown_content.select("ul.wds-list li a")]
        categories.extend(more_categories)

    # Удаляем элементы, соответствующие 'X more'
    categories = [cat for cat in categories if not re.match(r"^\d+\s+more$", cat)]
    
    # Удаляем дубликаты и очищаем категории
    return clean_categories(list(set(categories)))

    
def determine_class_and_subclass(content, infobox, categories):
    """Определяет основной класс и подкласс сущности."""
    entity_class = "Unclassified"
    subclass = categories.copy()  # Оригинальные категории сохраняются

    # Проверка для Character
    character_categories = [
        "Species Originating From Expanded Mythos Works",
        "Species Originating From Lovecraft Circle Works",
        "Species Originating From Mythos-Inspired Works",
        "Species Originating From Mythos-Adjacent Works",
        "Extra-Dimensional Species",
        "Extraterrestrial Species",
        "Extinct Species",
        "Earth-Native Species",
        "Non-Sapient Species",
        "Sapient Species",
        "Servitor Races",
        "Other Supernatural Beings",
        "Lifeforms & Entities",
        "Deep Ones",
        "Avatars of Azathoth",
        "Avatars of Cthulhu",
        "Avatars of Hastur",
        "Avatars of Nyarlathotep",
        "Avatars of Shub-Niggurath",
        "Avatars of Yog-Sothoth",
        "Avatars of Our Ladies of Sorrow",
        "Avatars",
        "Elder Gods",
        "Outer Gods",
        "Great Old Ones",
        "Great One",
        "Characters Originating From Expanded Mythos Works",
        "Deceased (In Mythos)",
        "Extraterrestrial Characters",
        "Extra-Dimensional Characters",
        "Humans", 
        "Characters Originating From Lovecraft Circle Works",
        "Characters Incorporated From Folklore",
        "Characters Originating From Mythos-Adjacent Works",
        "Characters Originating From Mythos-Inspired Works",
        "Magic Users",
        "Dreamers",
        "Species",
        "Disambiguations",
        "Deceased (In Mythos)"
    ]

    monster_keywords = ["creature", "god", "entity", "species", "race", "beast", "sapient", "humanoid", "extraterrestrial"]

    if any(cat in categories for cat in character_categories) or \
       ("species" in infobox and isinstance(infobox["species"], str) and any(monster_keyword in infobox["species"].lower() for monster_keyword in monster_keywords)) or \
       ("species" in infobox and isinstance(infobox["species"], list) and any(monster_keyword in item.lower() for item in infobox["species"] if isinstance(item, str) for monster_keyword in monster_keywords)):
        entity_class = "Character"
        return entity_class, subclass


    # Проверка для Artefact
    artefact_categories = [
        "Artefacts Originating From Expanded Mythos Works",
        "Artefacts Originating From Lovecraft Circle Works",
        "Artefacts Originating From Mythos-Inspired Works",
        "Artefacts Originating From Mythos-Adjacent Works",
        "Magical Artefacts", "Religious Artefacts",
        "Technological Artefacts", "Tools", "Weapons",
        "Books and Manuscripts", "Scrolls", "Ritual Objects",
        "Jewelry", "Keys", "Rings", "Scepters", "Mystical Artefacts", "Mythos Books (fictional)"
    ]

    artefact_keywords = [
        "book", "manuscript", "grimoire", "amulet", "stone", "device", "weapon",
        "scroll", "ring", "scepter", "artifact", "artefact", "key", "tool",
        "ritual object", "jewelry"
    ]

    if any(cat in categories for cat in artefact_categories) or \
       ("type" in infobox and isinstance(infobox["type"], str) and any(artefact_keyword in infobox["type"].lower() for artefact_keyword in artefact_keywords)) or \
       ("type" in infobox and isinstance(infobox["type"], list) and any(artefact_keyword in item.lower() for item in infobox["type"] if isinstance(item, str) for artefact_keyword in artefact_keywords)):
        entity_class = "Artefact"
        return entity_class, subclass

    # Проверка для Location
    location_categories = [
        "Locations", "Locations Originating From Expanded Mythos Works", "Locations Originating From Lovecraft Circle Works",
        "Locations Originating From Mythos-Inspired Works", "Locations Originating From Mythos-Adjacent Works",
        "Locations Incorporated From Folklore", "Locations Incorporated From the Real World", "Regions/Territories",
        "Extra-Dimensional Locations", "Extraterrestrial Locations", "Planets", "Stars", "Structures", "Dimensions"
    ]

    location_keywords = [
        "city", "region", "territory", "dimension", "plane", "realm",
        "mountain", "valley", "island", "forest", "lake", "river", "sea",
        "cave", "structure", "castle", "fortress", "ruins", "natural feature", "Iceberg-fortress"
    ]

    if any(cat in categories for cat in location_categories) or \
       ("type" in infobox and isinstance(infobox["type"], str) and any(location_keyword in infobox["type"].lower() for location_keyword in location_keywords)) or \
       ("type" in infobox and isinstance(infobox["type"], list) and any(location_keyword in item.lower() for item in infobox["type"] if isinstance(item, str) for location_keyword in location_keywords)):
        entity_class = "Location"
        return entity_class, subclass

    # Проверка для Organisations
    organisation_categories = [
        "Cults",
        "Defunct Organisations (fictional)",
        "Organisations",
        "Organisations Originating From Expanded Mythos Works",
        "Organisations Originating From Lovecraft Circle Works",
        "Organisations Originating From Mythos-Adjacent Works",
        "Organisations Originating From Mythos-Inspired Works"
    ]

    organisation_keywords = [
        "organisation", "group", "faction", "cult", "order", "association", "society"
    ]

    if any(cat in categories for cat in organisation_categories) or \
       ("type" in infobox and isinstance(infobox["type"], str) and any(organisation_keyword in infobox["type"].lower() for organisation_keyword in organisation_keywords)):
        entity_class = "Organisation"
        return entity_class, subclass

    # Проверка для RealWorldPerson
    real_person_categories = [
        "Expanded Mythos Authors", "Critics", "Artists", "Editors", "Scholars",
        "Deceased (Real World)", "Content Creators", "Publishers", "Traditional Games Designers",
        "Non-Fiction Authors", "Mythos-Inspired Authors", "Real World People",
        "H. P. Lovecraft's Correspondents", "Lovecraft Circle Authors",
        "Mythos Scholars", "Influences on Lovecraft's Fiction", "Games Designers",
        "Mythos-Adjacent Authors", "Alter-Egos/Characters Incorporated From the Real World", "Lovecraft's Inspirations (authors)",
        "Lovecraft's Correspondents", "Small Press"
    ]

    real_person_keywords = ["author", "editor", "scholar", "artist", "critic", "correspondent", "designer", "creator", "publisher", "historian", "biographer"]

    if any(cat in categories for cat in real_person_categories) or \
       any(key in infobox for key in ["birth_date", "death_date", "birthplace", "nationality"]):
        entity_class = "RealWorldPerson"
        return entity_class, subclass

    # Проверка для Work
    work_categories = [
        "H. P. Lovecraft works", "Non-Fiction Works", "Mythos-Inspired Works",
        "Mythos-Dedicated Anthologies", "Expanded Mythos Works", "Mythos-Adjacent Works",
        "Comic Books", "Periodicals", "Roleplaying Games", "Gaming Supplements",
        "Books", "Ebooks", "Anthologies", "Fanzines", "Story Cycles",
        "Novels", "Storybooks", "Arkham Horror Fiction", "Pulp Magazines",
        "Call of Cthulhu (real world)"
    ] + [cat for cat in categories if cat.lower().endswith("works")]

    if any(cat in categories for cat in work_categories) or \
       any(key in infobox for key in ["author", "publication_date", "language"]):
        if not any(cat in categories for cat in character_categories) and "species" not in infobox:
            entity_class = "Work"
            return entity_class, subclass

    return entity_class, subclass

def parse_offspring(value_list):
    """
    Обрабатывает данные о потомках, поддерживая разные форматы.
    """
    #logging.debug(f"Начало парсинга потомков: {value_list}")
    offspring = {key: [] for key in relative_keywords.values()}
    
    for item in value_list:
        #logging.debug(f"Обработка элемента: {item}")
        
        # Убираем ненужные префиксы и символы
        item = clean_prefixed_value(item)
        
        # Формат "Имя (Отношение)"
        matches_brackets = re.findall(r"(.+?)\s*\((.+?)\)", item.strip())
        
        if matches_brackets:
            for name, relation in matches_brackets:
                relation_key = relative_keywords.get(normalize_relation(relation), "hasOffspring")
                #logging.debug(f"Найдено соответствие: {name} - {relation} ({relation_key})")
                offspring[relation_key].append(name.strip())
        else:
            # Обработка строки без явных отношений (разделенной пробелами, запятыми или переносами строк)
            names = re.split(r"[\n,]+", item.strip())
            for name in names:
                clean_name = name.strip()
                if clean_name and not re.match(r"^(EXP|HPL|AWD|CIRCLE|ADJ)$", clean_name, re.IGNORECASE):
                    #logging.debug(f"Добавлено имя без отношения: {clean_name} как hasOffspring")
                    offspring["hasOffspring"].append(clean_name)
                else:
                    logging.warning(f"Пропущено некорректное значение: {clean_name}")
    
    # Удаляем пустые списки
    return {key: value for key, value in offspring.items() if value}


def parse_relatives(value_list):
    """
    Обрабатывает данные о родственниках, поддерживая следующие форматы:
    - Имя (Отношение)
    - Отношение: Имя
    - Несколько имен в одной строке, разделенных переносами или запятыми
    - Строки без указания отношения
    """
    #logging.debug(f"Начало парсинга родственников: {value_list}")
    
    # Словарь для хранения отношений и имён
    relatives = {}
    
    for item in value_list:
        item = clean_prefixed_value(item.strip())  # Очищаем от префиксов и лишних символов
        #logging.debug(f"Обработка очищенного элемента: {item}")
        
        # Формат "Имя (Отношение)"
        matches_brackets = re.findall(r"(.+?)\s*\((.+?)\)", item)
        # Формат "Отношение: Имя"
        matches_colon = re.findall(r"(\w+):\s*(.+)", item)
        
        if matches_brackets:
            for name, relation in matches_brackets:
                name = clean_prefixed_value(name.strip())  # Очистка имени
                relation = clean_prefixed_value(relation.strip())  # Очистка отношения
                relation_key = relative_keywords.get(normalize_relation(relation), "hasRelative")
                #logging.debug(f"Найдено соответствие (скобки): {name} - {relation} ({relation_key})")
                relatives.setdefault(relation_key, []).append(name)
        
        elif matches_colon:
            for relation, name in matches_colon:
                name = clean_prefixed_value(name.strip())  # Очистка имени
                relation = clean_prefixed_value(relation.strip())  # Очистка отношения
                relation_key = relative_keywords.get(normalize_relation(relation), "hasRelative")
                #logging.debug(f"Найдено соответствие (двоеточие): {relation} - {name} ({relation_key})")
                relatives.setdefault(relation_key, []).append(name)
        
        else:
            # Если формат не соответствует, разбиваем строки по \n или запятым
            names = re.split(r"[\n,]+", item)
            for name in names:
                name = clean_prefixed_value(name.strip())  # Очистка имени
                if name:
                    #logging.warning(f"Формат не соответствует ожиданиям: {name}, записано как hasRelative")
                    relatives.setdefault("hasRelative", []).append(name)
    
    # Удаляем пустые списки
    return {key: value for key, value in relatives.items() if value}


def extract_aliases(value):
    """
    Извлекает алиасы, включая текст, ссылки и преобразует ссылки в названия статей.
    """
    soup = BeautifulSoup(value, "html.parser")
    aliases = []

    for elem in soup.contents:
        if elem.name == "a":  # Если это ссылка
            # Извлекаем title из атрибута ссылки
            if "title" in elem.attrs:
                aliases.append(elem["title"].strip())
        elif isinstance(elem, str):  # Если это текст
            # Разделяем текст по запятым
            aliases.extend([alias.strip() for alias in elem.split(",") if alias.strip()])

    return aliases

def parse_born(value):
    """
    Парсит значение 'born', извлекая source, location и additional_info по заданным правилам,
    с поддержкой множественных блоков.
    """
    soup = BeautifulSoup(value, "html.parser")
    text = soup.get_text(separator=" ", strip=True)  # Извлекаем весь текст
    tokens = re.split(r"(?<=:)", text)  # Разделяем текст по двоеточию, сохраняя его

    result = []  # Список для всех найденных блоков
    current_block = {
        "source": None,
        "location": None,
        "additional_info": None
    }

    for token in tokens:
        # Удаляем лишние пробелы
        token = token.strip()
        
        # Если это источник (source)
        if token in ["HPL:", "AWD:", "CIRCLE:", "EXP:", "INFL:", "ADJ:"]:
            # Если текущий блок заполнен, добавляем его в результат
            if current_block["source"] or current_block["location"] or current_block["additional_info"]:
                result.append(current_block)
                current_block = {"source": None, "location": None, "additional_info": None}
            
            # Устанавливаем source
            current_block["source"] = token.rstrip(":")
        else:
            # Если есть запятая, делим на location и additional_info
            if "," in token:
                location, additional_info = map(str.strip, token.split(",", 1))
                current_block["location"] = location
                current_block["additional_info"] = additional_info
            else:
                # Если запятой нет, это location
                current_block["location"] = token

    # Добавляем последний блок, если он непустой
    if current_block["source"] or current_block["location"] or current_block["additional_info"]:
        result.append(current_block)

    return result
    
def prepare_infobox_text(infobox_elem):
    """
    Извлекает данные из инфобокса и обрабатывает значения, чтобы поддерживать пробелы.
    """
    infobox_data = []
    for data_block in infobox_elem.find_all("div", class_="pi-data"):
        label = data_block.find("h3", class_="pi-data-label")
        value = data_block.find("div", class_="pi-data-value")
        if label and value:
            raw_text = value.get_text(separator=" ", strip=False)
            
            # Обрабатываем поле "born" отдельно
            if label.get_text(strip=True).lower() == "born":
                processed_value = [raw_text]  # Сохраняем в список для согласованности
                infobox_data.append({"label": label.get_text(strip=True), "value": processed_value})
            else:
                infobox_data.append({"label": label.get_text(strip=True), "value": str(value)})
    return infobox_data


def split_field(field_value, delimiters=["&", " and ", ",", "/", ";", "\n"]):
    """
    Разделяет строку на отдельные значения по указанным разделителям.
    Учитывает исключения для сайтов (URL-адресов), удаляет 'and', '&', и обрабатывает суффиксы (Jr., Sr. и т.п.).
    """
    if isinstance(field_value, str):
        #print(f"Исходное значение: {field_value}")
        
        # Заменяем запятые перед суффиксами на точки
        field_value = re.sub(r",\s*(Jr\.|Sr\.|III|IV|V)", r". \1", field_value)
        #print(f"После замены запятых перед суффиксами: {field_value}")
        
        # Проверяем на наличие URL-адресов и исключаем их из разбиения
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        urls = re.findall(url_pattern, field_value)
        if urls:
            for url in urls:
                field_value = field_value.replace(url, "")
        #print(f"После удаления URL-адресов: {field_value}")
        #print(f"Найденные URL-адреса: {urls}")
        
        # Шаг 1: Разбиение по основным разделителям
        pattern = "|".join(map(re.escape, delimiters))
        temp_split = [
            item.strip() for item in re.split(pattern, field_value) 
            if item.strip() and item.strip().lower() not in ["and", "&"]
        ]
        #print(f"После разбиения по основным разделителям: {temp_split}")
        
        # Шаг 2: Обработка суффиксов и запятых
        final_split = []
        for item in temp_split:
            # Если строка содержит суффикс, оставляем её как есть
            if re.search(r"\b(Jr\.|Sr\.|III|IV|V)$", item):
                final_split.append(item.strip())
            else:
                split_items = [sub.strip() for sub in item.split(",") if sub.strip()]
                final_split.extend(split_items)
        
        # Добавляем обратно URL-адреса
        final_split.extend(urls)
        #print(f"Результат разбиения: {final_split}")
        
        return final_split
    return field_value

def replace_na(value):
    if isinstance(value, str):
        return "Unknown" if value.strip().upper() in ["N/A", "NA", "UNKNOWN"] else value
    elif isinstance(value, list):
        return [replace_na(v) for v in value]
    return value

    
def clean_html(raw_html):
    """
    Очищает HTML, извлекая текстовое содержимое и полностью удаляя <small> с его содержимым.
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    
    # Удаляем все теги <small> с их содержимым
    for small in soup.find_all("small"):
        small.decompose()
    
    # Обрабатываем текст, заменяя <br> на разрывы строк
    result = []
    for element in soup.descendants:
        if element.name == "br":
            result.append("\n")  # Разделяем строки
        elif isinstance(element, str) and element.strip():
            result.append(element.strip())

    return " ".join(result).replace("\n ", "\n").strip()


def extract_entities_from_html(value):
    """
    Извлекает сущности из HTML. Если есть несколько <a> или <br>, возвращает список.
    """
    soup = BeautifulSoup(value, "html.parser")
    entities = []

    for element in soup.descendants:
        if element.name == "a":  # Ссылки
            entities.append(element.get_text(strip=True))
        elif element.name == "br":  # Разделение по <br>
            continue
        elif isinstance(element, str) and element.strip() not in ["and", "&"]:  # Чистый текст
            entities.append(element.strip())

    # Убираем дубликаты и пустые элементы
    return list(dict.fromkeys(entities)) if entities else ""

name_mapping = {
    "Howard Phillips Lovecraft": "H. P. Lovecraft",
    "H. P. Lovecraft": "H. P. Lovecraft",
    "Lovecraft, H. P.": "H. P. Lovecraft",
    "H.P. Lovecraft": "H. P. Lovecraft",
    "Howard P. Lovecraft": "H. P. Lovecraft",
    "H.P.L.": "H. P. Lovecraft",
}

def normalize_name(name):
    """
    Нормализует имя на основе маппинга.
    """
    return name_mapping.get(name.strip(), name)
    
def normalize_value(value):
    """
    Проверяет и нормализует значение, заменяя псевдонимы Говарда Лавкрафта
    на стандартное имя 'H. P. Lovecraft'.
    """
    if isinstance(value, str):
        return normalize_name(value)  # Нормализация строки
    elif isinstance(value, list):
        return [normalize_name(v) if isinstance(v, str) else v for v in value]  # Нормализация списка
    return value  # Возвращаем значение без изменений, если это не строка и не список
    
def clean_prefixed_value(value):
    if isinstance(value, str):
        # Очищаем строки
        value = re.sub(r'^(EXP|HPL|CIRCLE|AWD|ADJ|INFL)\s*:\s*', '', value)
        value = value.replace('&quot;', '').replace('"', '').strip()
        return value
    elif isinstance(value, list):
        # Рекурсивно очищаем каждый элемент
        return [clean_prefixed_value(v) for v in value]
    elif isinstance(value, dict):
        # Если значение — словарь, очищаем его ключи и значения
        return {k: clean_prefixed_value(v) for k, v in value.items()}
    return value
    
def parse_infobox(prepared_infobox_data):
    """
    Обрабатывает инфобокс, извлекая данные, включая ссылки, текст и разрывы строк.
    """
    extracted_data = {}
    
    for item in prepared_infobox_data:
        field = item["label"].lower().replace(" ", "_").strip()
        raw_value = item["value"]

        # Если значение — список, преобразуем его элементы в текст
        if isinstance(raw_value, list):
            value = [clean_html(str(v)) for v in raw_value]
        else:
            value = clean_html(raw_value)

        # Применяем замену N/A на Unknown
        value = replace_na(value)

        # Очищаем значение от префиксов и кавычек
        value = clean_prefixed_value(value)

        # Нормализуем значение
        normalized_value = normalize_value(value)
        
        # Поля, которые требуют специальной обработки
        if field == "author":
            # Разделяем авторов и нормализуем
            authors = split_field(value, delimiters=[" and ", "&", ",", ";", "/", "\n"])
            extracted_data[field] = [normalize_name(author) for author in authors]
        elif field in ["appearances", "aliases", "titles"]:
            extracted_data[field] = split_field(normalized_value, delimiters=["\n"])
        elif field == "created_by":
            # Нормализуем поле 'created_by'
            if isinstance(normalized_value, list):
                extracted_data[field] = [normalize_name(v) for v in normalized_value]
            else:
                extracted_data[field] = normalize_name(normalized_value)
        elif field in ["publication", "birth_date", "death_date", "birthplace", "deathplace", "publication_date", "release_date"]:
            if isinstance(normalized_value, list):
                extracted_data[field] = str(normalized_value)
        elif field in ["website", "url"]:
            extracted_entities = extract_entities_from_html(raw_value)
            if isinstance(extracted_entities, list):
                extracted_data[field] = extracted_entities[0] if extracted_entities else ""
            else:
                extracted_data[field] = extracted_entities

        if field == "relatives":
            # Обрабатываем relatives и добавляем отдельные поля
            relatives = parse_relatives(normalized_value if isinstance(normalized_value, list) else [normalized_value])
            extracted_data.update(relatives)
        elif field == "offspring":
            # Обработка offspring аналогично
            offspring = parse_offspring(normalized_value if isinstance(normalized_value, list) else [normalized_value])
            extracted_data.update(offspring)
        else:
            # Универсальная обработка остальных полей
            extracted_data[field] = split_field(normalized_value)

    return extracted_data

def extract_clean_content(soup, infobox_data, title):
    """
    Извлекает и очищает контент статьи, исключая данные инфобокса и ненужные элементы, но оставляя текстовые данные.
    """
    content_div = soup.find("div", class_="mw-parser-output")
    if not content_div:
        return ""

    # Извлекаем текст из всех подходящих элементов
    content = []
    for element in content_div.find_all(["p", "h2", "h3", "ul", "ol"], recursive=False):
        if element.find_parent("aside"):
            continue  # Пропускаем инфобокс
        content.append(element.get_text(" ", strip=True))

    raw_content = "\n\n".join(content).strip()

    # Удаляем только ключевые элементы инфобокса (метки, такие как "Author", "Illustrator" и т.д.)
    for key in infobox_data.keys():
        raw_content = re.sub(rf"\b{re.escape(key)}\b", "", raw_content, flags=re.IGNORECASE)

    # Удаляем заголовок статьи из текста
    if title:
        raw_content = re.sub(rf"\b{re.escape(title)}\b", "", raw_content, flags=re.IGNORECASE)

    # Финальная чистка текста
    clean_content = re.sub(r"\s{2,}", " ", raw_content).strip()
    clean_content = re.sub(r"\[.*?\]", "", clean_content)  # Удаляем [ссылки]
    clean_content = re.sub(r",\s*,", ",", clean_content)  # Лишние запятые
    clean_content = re.sub(r"^\W+", "", clean_content)  # Убираем символы в начале строки

    return clean_content


def parse_article(article_url):
    """
    Парсит статью и возвращает данные.
    """
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, "html.parser")

    title = soup.find("h1", class_="page-header__title").text.strip() if soup.find("h1") else "Unknown Title"

    # Извлекаем инфобокс
    infobox_elem = soup.find("aside", class_="portable-infobox")
    parsed_infobox = parse_infobox(prepare_infobox_text(infobox_elem)) if infobox_elem else {}

    # Извлекаем контент
    content = extract_clean_content(soup, parsed_infobox, title)

    # Классификация
    categories = extract_top_categories(soup)
    entity_class, subclass = determine_class_and_subclass(content, parsed_infobox, categories)

    return {
        "title": title,
        "class": entity_class,
        "subclass": subclass,
        "content": content.strip(),
        "infobox": parsed_infobox,
    }

def extract_top_categories(soup):
    categories_elem = soup.find("div", class_="page-header__categories")
    if categories_elem:
        return clean_categories([link.text.strip() for link in categories_elem.find_all("a")])
    return []

def get_next_page_from(soup):
    next_button = soup.find("a", string=lambda text: text and text.startswith("Next page"))
    if next_button:
        next_article_name = next_button.text.replace("Next page", "").strip(" ()")
        return f"{BASE_URL}/wiki/Special:AllPages?from={urllib.parse.quote(next_article_name)}"
    return None

def parse_all_articles(start_page, max_pages=18, workers=350):
    articles_data, current_page, page_count = [], start_page, 1

    with ThreadPoolExecutor(max_workers=workers) as executor:
        while current_page and page_count <= max_pages:
            logging.info(f"Парсим страницу {page_count}: {current_page}")
            soup = BeautifulSoup(requests.get(current_page).text, "html.parser")

            links = [{"name": link.text.strip(), "url": BASE_URL + link["href"]} for link in soup.select("ul.mw-allpages-chunk a")]
            future_results = executor.map(parse_article, [article["url"] for article in links])

            articles_data.extend(future_results)
            current_page = get_next_page_from(soup)
            page_count += 1

    return articles_data

def save_to_json(data, filename="transformed_data.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        logging.info(f"Данные сохранены в файл {filename}")

def main():
    start_page = f"{BASE_URL}/wiki/Special:AllPages"
    articles_data = parse_all_articles(start_page)
    save_to_json(articles_data)

if __name__ == "__main__":
    main()

2025-01-14 10:27:35 - INFO - Парсим страницу 1: https://lovecraft.fandom.com/wiki/Special:AllPages
/var/folders/tz/03m96h9d79gb8ybshv14drgw0000gn/T/ipykernel_61379/2346216578.py:477: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(raw_html, "html.parser")
2025-01-14 10:27:52 - INFO - Парсим страницу 2: https://lovecraft.fandom.com/wiki/Special:AllPages?from=Arthur%20Jermyn
2025-01-14 10:28:07 - INFO - Парсим страницу 3: https://lovecraft.fandom.com/wiki/Special:AllPages?from=Call%20of%20Cthulhu%3A%20Arkham
2025-01-14 10:28:23 - INFO - Парсим страницу 4: https://lovecraft.fandom.com/wiki/Special:AllPages?from=Crypt%20of%20Cthulhu%20Roodmas%201982
2025-01-14 10:28:40 - INFO - Парсим страницу 5: https://lovecraft.fandom.com/wiki/Special:AllPages?from=Dead%20but%20Dreaming%202
2025-01-14 10:28:54 - INFO - Парсим страницу 6: https://lovecraft.fandom.com/wiki/Sp